# Property-based testing

In [32]:
from functools import reduce
from typing import Tuple, Dict

### Property-Based Testing with Hypothesis


In [33]:
# Before continuing, install and import the 'hypothesis' library

"""
!pip install hypothesis
"""
from hypothesis import given, example
from hypothesis.strategies import integers, text, dictionaries, composite
from hypothesis.stateful import Bundle, RuleBasedStateMachine, invariant, rule


In [34]:
# Consider the following example-based unit test

def test_meal_recommendation_under_specific_calories():
    calories = 900
    meals = get_recommended_meal(Recommendation.BY_CALORIES, calories)
    assert meals == [
        Meal("Spring Roll", 120),
        Meal("Green Papaya Salad", 230),
        Meal("Larb Chicken", 500)
    ]


In [35]:
# Instead of testing specific examples, we can test properties 
# about our system, as follows:

def test_meal_recommendation_under_specific_calories():
    calories = 900
    meals = get_recommended_meal(Recommendation.BY_CALORIES, calories)
    assert len(meals) == 3
    assert is_appetizer(meals[0])
    assert is_salad(meals[1])
    assert is_main_dish(meals[2])
    assert sum(meal.calories for meal in meals) < calories


In [36]:
# In the last test, we can use the 'given' decorator from hypothesis 
# to generate input data as integers, as follows:

@given(integers())
def test_meal_recommendation_under_specific_calories(calories):
    meals = get_recommended_meal(Recommendation.BY_CALORIES, calories)
    assert len(meals) == 3
    assert is_appetizer(meals[0])
    assert is_salad(meals[1])
    assert is_main_dish(meals[2])
    assert sum(meal.calories for meal in meals) < calories


In [37]:
# We can also specify the minimum value of integers, as follows:

@given(integers(min_value=900))
def test_meal_recommendation_under_specific_calories(calories):
    meals = get_recommended_meal(Recommendation.BY_CALORIES, calories)
    assert len(meals) == 3
    assert is_appetizer(meals[0])
    assert is_salad(meals[1])
    assert is_main_dish(meals[2])
    assert sum(meal.calories for meal in meals) < calories


In [38]:
# We can test specific cases with the 'example' decorator, as follows:

@example(5001)
@given(integers(min_value=900))
def test_meal_recommendation_under_specific_calories(calories):
    meals = get_recommended_meal(Recommendation.BY_CALORIES, calories)
    assert len(meals) == 3
    assert is_appetizer(meals[0])
    assert is_salad(meals[1])
    assert is_main_dish(meals[2])
    assert sum(meal.calories for meal in meals) < calories



### Getting the Most Out of Hypothesis

Hypothesis Strategies


In [39]:
# We can create strategies that compose other strategies together, 
# such as building dictionaries of strategies:

@given(dictionaries(text(), integers(min_value=100, max_value=2000)))
def test_calorie_count(ingredient_to_calorie_mapping : Dict[str, int]):
    # ... snip ...
    return


In [40]:
# For example, to create a strategy that creates three-course meals,
# we can use the 'composite' decorator to define a custom strategy, as follows:

class Dish:
    pass

ThreeCourseMeal = Tuple[Dish, Dish, Dish]

@composite
def three_course_meals(draw) -> ThreeCourseMeal:
    appetizer_calories = integers(min_value=100, max_value=900)
    main_dish_calories = integers(min_value=550, max_value=1800)
    dessert_calories = integers(min_value=500, max_value=1000)
    return (
        Dish("Appetizer", draw(appetizer_calories)),
        Dish("Main Dish", draw(main_dish_calories)),
        Dish("Dessert", draw(dessert_calories))
    )

@given(three_course_meals)
def test_three_course_meal_substitutions(three_course_meal: ThreeCourseMeal):
    # ... do something with three_course_meal
    return


Generating Algorithms

Consider the following properties to assert about the system in a meal recommendation system:
```
• The meal recommendation system always returns 3 meal options.
• All 3 meal options are unique.
• The meal options are ordered based on the most recent filter applied. In the case of ties, the next most recent filter is used.
• New filters replace old filters of the same type. 
```

In [41]:
# Instead of writing lots of test cases, we can represent the last properties 
# with a class derived from 'RuleBasedStateMachine', as follows: 

class MealRecommendationEngine():
    pass

class RecommendationChecker(RuleBasedStateMachine):
    def __init__(self):
        super().__init__()
        self.recommender = MealRecommendationEngine()
        self.filters = []

    @rule(price_limit=integers(min_value=6, max_value=200))
    def filter_by_price(self, price_limit):
        self.recommender.apply_price_filter(price_limit)
        self.filters = [f for f in self.filters if f[0] != "price"]
        self.filters.append(("price", lambda m: m.price))

    @rule(calorie_limit=integers(min_value=500, max_value=2000))
    def filter_by_calories(self, calorie_limit):
        self.recommender.apply_calorie_filter(calorie_limit)
        self.filters = [f for f in self.filters if f[0] != "calorie"]
        self.filters.append(("calorie", lambda m: m.calories))

    @rule(distance_limit=integers(max_value=100))
    def filter_by_distance(self, distance_limit):
        self.recommender.apply_distance_filter(distance_limit)
        self.filters = [f for f in self.filters if f[0] != "distance"]
        self.filters.append(("distance", lambda m: m.distance))

    @invariant()
    def recommender_provides_three_unique_meals(self):
        assert len(self.recommender.get_meals()) == 3
        assert len(set(self.recommender.get_meals())) == 3

    @invariant()
    def meals_are_appropriately_ordered(self):
        meals = self.recommender.get_meals()
        ordered_meals = reduce(
            lambda meals, f: sorted(meals, key=f[1]),
            self.filters,
            meals
        )
        assert ordered_meals == meals


In [42]:
# Then save the test class in a variable prefixed with 'Test',
# so pytest can discover the stateful Hypothesis test:

TestRecommender = RecommendationChecker.TestCase
